# Symbolic regression of a dynamical system

In this example, Kozax is applied to recover the state equations of the Lotka-Volterra system. The candidate solutions are integrated as a system of differential equations, after which the predictions are compared to the true observations to determine a fitness score.

In [ ]:
# Specify the cores to use for XLA
import os
os.environ["XLA_FLAGS"] = '--xla_force_host_platform_device_count=10'

import jax
import diffrax
import jax.numpy as jnp
import jax.random as jr
import diffrax

from kozax.genetic_programming import GeneticProgramming
from kozax.fitness_functions.ODE_fitness_function import ODEFitnessFunction
from kozax.environments.SR_environments.lotka_volterra import LotkaVolterra

These device(s) are detected:  [CpuDevice(id=0), CpuDevice(id=1), CpuDevice(id=2), CpuDevice(id=3), CpuDevice(id=4), CpuDevice(id=5), CpuDevice(id=6), CpuDevice(id=7), CpuDevice(id=8), CpuDevice(id=9)]


First the data is generated, consisting of initial conditions, time points and the true observations. Kozax provides the Lotka-Volterra environment, which is integrated with Diffrax.

In [9]:
def get_data(key, env, dt, T, batch_size=20):
    x0s = env.sample_init_states(batch_size, key)
    ts = jnp.arange(0, T, dt)

    def solve(env, ts, x0):
        solver = diffrax.Dopri5()
        dt0 = 0.001
        saveat = diffrax.SaveAt(ts=ts)

        system = diffrax.ODETerm(env.drift)

        # Solve the system given an initial conditions
        sol = diffrax.diffeqsolve(system, solver, ts[0], ts[-1], dt0, x0, saveat=saveat, max_steps=500, 
                                  adjoint=diffrax.DirectAdjoint(), stepsize_controller=diffrax.PIDController(atol=1e-7, rtol=1e-7, dtmin=0.001))
        
        return sol.ys

    ys = jax.vmap(solve, in_axes=[None, None, 0])(env, ts, x0s) #Parallelize over the batch dimension
    
    return x0s, ts, ys

key = jr.PRNGKey(0)
data_key, gp_key = jr.split(key)

T = 30
dt = 0.2
env = LotkaVolterra()

# Simulate the data
data = get_data(data_key, env, dt, T, batch_size=4)
x0s, ts, ys = data

For the fitness function, we used the ODEFitnessFunction that uses Diffrax to integrate candidate solutions. It is possible to select the solver, time step, number of steps and a stepsize controller to balance efficiency and accuracy. To ensure convergence of the genetic programming algorithm, constant optimization is applied to the best candidates at every generation. The constant optimization is performed with a couple of simple evolutionary steps that adjust the values of the constants in a candidate. The hyperparameters that define the constant optimization are `constant_optimization_N_offspring` (number of candidates with different constants should be sampled for each candidate), `constant_optimization_steps` (number of iterations of constant optimization for each candidate), `optimize_constants_elite` (number of candidates that constant optimization is applied to), `constant_step_size_init` (initial value of the step size for sampling constants) and `constant_step_size_decay` (the rate of decrease of the step size over generations).

In [10]:
#Define the nodes and hyperparameters
operator_list = [
        ("+", lambda x, y: jnp.add(x, y), 2, 0.5), 
        ("-", lambda x, y: jnp.subtract(x, y), 2, 0.1), 
        ("*", lambda x, y: jnp.multiply(x, y), 2, 0.5), 
    ]

variable_list = [["x" + str(i) for i in range(env.n_var)]]
layer_sizes = jnp.array([env.n_var])

population_size = 100
num_populations = 10
num_generations = 50

#Initialize the fitness function and the genetic programming strategy
fitness_function = ODEFitnessFunction(solver=diffrax.Dopri5(), dt0 = 0.01, stepsize_controller=diffrax.PIDController(atol=1e-6, rtol=1e-6, dtmin=0.001), max_steps=300)

strategy = GeneticProgramming(num_generations, population_size, fitness_function, operator_list, variable_list, layer_sizes, num_populations = num_populations,
                        size_parsimony=0.003, constant_optimization_method="evolution", constant_optimization_N_offspring = 25, constant_optimization_steps = 5, 
                        optimize_constants_elite=100, constant_step_size_init=0.1, constant_step_size_decay=0.99)

Input data should be formatted as: ['x0', 'x1'].


Kozax provides a fit function that receives the data and a random key. However, it is also possible to run Kozax with an easy loop consisting of evaluating and evolving. This is useful as different input data can be provided during evaluation. In symbolic regression of dynamical systems, it helps to first optimize on a small part of the time points, and provide the full data trajectories only after a couple of generations.

In [11]:
# Sample the initial population
population = strategy.initialize_population(gp_key)

# Define the number of timepoints to include in the data
end_ts = int(ts.shape[0]/2)

for g in range(num_generations):
    if g == 25: # After 25 generations, use the full data
        end_ts = ts.shape[0]

    key, eval_key, sample_key = jr.split(key, 3)
    # Evaluate the population on the data, and return the fitness
    fitness, population = strategy.evaluate_population(population, (x0s, ts[:end_ts], ys[:,:end_ts]), eval_key)

    # Print the best solution in the population in this generation
    best_fitness, best_solution = strategy.get_statistics(g)
    print(f"In generation {g+1}, best fitness = {best_fitness:.4f}, best solution = {strategy.expression_to_string(best_solution)}")

    # Evolve the population until the last generation. The fitness should be given to the evolve function.
    if g < (num_generations-1):
        population = strategy.evolve_population(population, fitness, sample_key)

In generation 1, best fitness = 0.8547, best solution = [-0.414*x0*x1 + x0, 0.446*x0 - 0.377*x1]
In generation 2, best fitness = 0.8428, best solution = [-0.381*x0*x1 + x0, 0.422*x0 - 0.361*x1]
In generation 3, best fitness = 0.8428, best solution = [-0.381*x0*x1 + x0, 0.422*x0 - 0.361*x1]
In generation 4, best fitness = 0.8397, best solution = [-0.397*x0*x1 + x0, 0.374*x0 - 0.356*x1]
In generation 5, best fitness = 0.7787, best solution = [-1.28*x0*(0.302*x1 - 0.888), 0.315*x0 - 0.21*x1 - 0.391]
In generation 6, best fitness = 0.6935, best solution = [-1.54*x0*(0.293*x1 - 0.819), 0.285*x0 - 0.233*x1 - 0.358]
In generation 7, best fitness = 0.6935, best solution = [-1.54*x0*(0.293*x1 - 0.819), 0.285*x0 - 0.233*x1 - 0.358]
In generation 8, best fitness = 0.6689, best solution = [-1.68*x0*(0.305*x1 - 0.793), 0.244*x0 - 0.233*x1 - 0.354]
In generation 9, best fitness = 0.6648, best solution = [-1.36*x0*(0.35*x1 - 0.989), 0.238*x0 - 0.22*x1 - 0.343]
In generation 10, best fitness = 0.6648,

In [12]:
strategy.print_pareto_front()

Complexity: 2, fitness: 3.603614091873169, equations: [-0.0703, -0.395]
Complexity: 4, fitness: 2.3969385623931885, equations: [-1.09*x0, -0.715]
Complexity: 6, fitness: 1.2524828910827637, equations: [-1.38*x0, -0.327*x1]
Complexity: 8, fitness: 1.1349678039550781, equations: [-2.74*x0, x0 - 0.432*x1]
Complexity: 10, fitness: 1.1001994609832764, equations: [-2.91*x0, x0 - 0.427*x1 + 0.0944]
Complexity: 12, fitness: 1.0081340074539185, equations: [-0.44*x0*(x0 + x1 - 3.0), -0.318*x1]
Complexity: 14, fitness: 0.749535083770752, equations: [-0.402*x0*x1 + x0, 0.229*x0 - 0.366*x1]
Complexity: 16, fitness: 0.6201074123382568, equations: [-0.449*x0*(x1 - 2.82), 0.0934*x0*x1 - 0.382*x1]
Complexity: 18, fitness: 0.08289898931980133, equations: [-0.408*x0*(x1 - 2.82), 0.0934*x0*x1 - 0.382*x1]
Complexity: 20, fitness: 0.033442918211221695, equations: [-0.407*x0*(x1 - 2.74), 0.102*x0*x1 - 0.401*x1]


Instead of using evolution to optimize the constants, Kozax also offers gradient-based optimization. For gradient optimization, it is possible to specify the optimizer, the number of candidates to apply constant optimization to, the initial learning rate and the learning rate decay over generation. These two methods are provided as either can be more effective or efficient for different problems.

In [6]:
import optax

strategy = GeneticProgramming(num_generations, population_size, fitness_function, operator_list, variable_list, layer_sizes, num_populations = num_populations,
                        size_parsimony=0.003, constant_optimization_method="gradient", constant_optimization_steps = 15, optimizer_class = optax.adam,
                        optimize_constants_elite=100, constant_step_size_init=0.01, constant_step_size_decay=0.99)

Input data should be formatted as: ['x0', 'x1'].


In [7]:
key = jr.PRNGKey(0)
data_key, gp_key = jr.split(key)

T = 30
dt = 0.2
env = LotkaVolterra()

# Simulate the data
data = get_data(data_key, env, dt, T, batch_size=4)
x0s, ts, ys = data

# Sample the initial population
population = strategy.initialize_population(gp_key)

# Define the number of timepoints to include in the data
end_ts = int(ts.shape[0]/2)

for g in range(num_generations):
    if g == 25: # After 25 generations, use the full data
        end_ts = ts.shape[0]

    key, eval_key, sample_key = jr.split(key, 3)
    # Evaluate the population on the data, and return the fitness
    fitness, population = strategy.evaluate_population(population, (x0s, ts[:end_ts], ys[:,:end_ts]), eval_key)

    # Print the best solution in the population in this generation
    best_fitness, best_solution = strategy.get_statistics(g)
    print(f"In generation {g+1}, best fitness = {best_fitness:.4f}, best solution = {strategy.expression_to_string(best_solution)}")

    # Evolve the population until the last generation. The fitness should be given to the evolve function.
    if g < (num_generations-1):
        population = strategy.evolve_population(population, fitness, sample_key)

In generation 1, best fitness = 0.8637, best solution = [-0.394*x0*x1 + x0, 0.529*x0 - 0.367*x1]
In generation 2, best fitness = 0.8579, best solution = [-0.393*x0*x1 + x0, 0.525*x0 - 0.372*x1]
In generation 3, best fitness = 0.7924, best solution = [-0.407*x0*x1 + x0, 0.185*x0 - 0.36*x1]
In generation 4, best fitness = 0.7923, best solution = [-0.403*x0*x1 + x0, 0.182*x0 - 0.357*x1]
In generation 5, best fitness = 0.7918, best solution = [-0.402*x0*x1 + x0, 0.182*x0 - 0.359*x1]
In generation 6, best fitness = 0.7913, best solution = [-0.403*x0*x1 + x0, 0.184*x0 - 0.359*x1]
In generation 7, best fitness = 0.7913, best solution = [-0.403*x0*x1 + x0, 0.184*x0 - 0.359*x1]
In generation 8, best fitness = 0.7913, best solution = [-0.403*x0*x1 + x0, 0.184*x0 - 0.359*x1]
In generation 9, best fitness = 0.7913, best solution = [-0.403*x0*x1 + x0, 0.184*x0 - 0.359*x1]
In generation 10, best fitness = 0.7913, best solution = [-0.403*x0*x1 + x0, 0.184*x0 - 0.359*x1]
In generation 11, best fitness

In [8]:
strategy.print_pareto_front()

Complexity: 2, fitness: 3.4025232791900635, equations: [-0.193, -0.455]
Complexity: 4, fitness: 2.334573268890381, equations: [-1.38*x0, -0.743]
Complexity: 6, fitness: 1.192003607749939, equations: [-4.32*x0, -0.328*x1]
Complexity: 8, fitness: 1.1891828775405884, equations: [-0.482*x0*x1, -0.33*x1]
Complexity: 10, fitness: 1.099962830543518, equations: [-2.78*x0, x0 - 0.427*x1 + 0.0845]
Complexity: 12, fitness: 0.8712133169174194, equations: [-0.396*x0*x1 + x0, x0 - 0.398*x1]
Complexity: 14, fitness: 0.7701644897460938, equations: [-0.394*x0*x1 + x0, 0.318*x0 - 0.361*x1]
Complexity: 16, fitness: 0.27488815784454346, equations: [-0.343*x0*x1 + x0, 0.0965*x0*x1 - 0.394*x1]
Complexity: 18, fitness: 0.007773575372993946, equations: [-0.399*x0*(x1 - 2.75), 0.101*x0*x1 - 0.401*x1]
Complexity: 22, fitness: 0.003365457523614168, equations: [-0.4*x0*(x1 - 2.75), 0.0998*x0*x1 - 0.399*x1]
